In [13]:
## BIBLIOTECAS
from urllib.parse import urlparse
import mysql.connector
import pandas as pd
import os

## VARIAVEIS

## CAMINHOS
path_planilha = r'C:\Users\mathe\OneDrive\Área de Trabalho\DESPESAS_MENSAIS.xlsx'

## DADOS BASE PARA O MYSQL
# LEITURA DA PLANILHAS
df = pd.read_excel(path_planilha, sheet_name='BD_DESPESAS')

df = df.rename(columns={
    'CONTA':'DS_DESPESA',
    'R$':'VR_DESPESA',
    'DATA VENCIMENTO':'DT_VENCIMENTO',
    'ANO VENCIMENTO':'DT_ANO_VECIMENTO',
    'MÊS VENCIMENTO':'DT_MES_VECIMENTO',
    'ATRASO':'QT_DIAS_ATRASO',
    'STATUS':'ST_VENCIMENTO',
    'DATA DE PAMENTO':'DT_PAGAMENTO',
    'ANO PAGAMENTO':'DT_ANO_PAGAMENTO',
    'MÊS PAGAMENTO':'DT_MES_PAGAMENTO',
    'DIA PAGAMENTO':'DT_DIA_PAGAMENTO',
    'PERIODO':'DT_PERIODO',
    'TIPO':'ST_PAGAMENTO',
    'QUEM':'NM_PAGADOR'
})

## CONEXAO MYSQL
# LEITURA DO ARQUIVO .ENV
database_url = os.getenv("DATABASE_URL")

# COMPOR A URL DE CONEXAO
url_components = urlparse(database_url)
conn = mysql.connector.connect(
    host=url_components.hostname,
    port=url_components.port,
    user=url_components.username,
    password=url_components.password,
    database=url_components.path[1:]
)

# ABERTURA DO CURSO
curso = conn.cursor()

# FUNCAO PARA CRIAR TABELA NO MYSQL
def CriarTabelaDoZero(df, nome_tabela, database_url):
    df.to_sql(nome_tabela, database_url, index=False, if_exists='replace')

# FUNCAO PARA APPENDAR NOVOS DADOS NA TABELA
def AppendNovosDados(df, nome_tabela, database_url):
    # LER A TABELA NO MYSQL
    df_sql = pd.read_sql(nome_tabela, con=database_url)
    # CRIAR O LEFTANTI DO BANCO PARA A PLANILHA
    merge_leftanti = df.merge(df_sql, on=['DS_DESPESA', 'VR_DESPESA', 'DT_VENCIMENTO'], how='left', indicator=True)
    df_geral = merge_leftanti[merge_leftanti['_merge'] == 'left_only']
    #APPENDAR OS DADOS NO MYSQL
    df_geral.to_sql(nome_tabela, database_url, if_exists='append', index=False)

c:\00_Dev\App-Despesas-TM\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [14]:
# CriarTabelaDoZero(df=df, nome_tabela='tbl_despesas', database_url=database_url)

In [17]:
df_geral

,DS_DESPESA,VR_DESPESA,DT_VENCIMENTO,DT_ANO_VECIMENTO_x,DT_MES_VECIMENTO_x,QT_DIAS_ATRASO_x,ST_VENCIMENTO_x,DT_PAGAMENTO_x,DT_ANO_PAGAMENTO_x,DT_MES_PAGAMENTO_x,...,QT_DIAS_ATRASO_y,ST_VENCIMENTO_y,DT_PAGAMENTO_y,DT_ANO_PAGAMENTO_y,DT_MES_PAGAMENTO_y,DT_DIA_PAGAMENTO_y,DT_PERIODO_y,ST_PAGAMENTO_y,NM_PAGADOR_y,_merge
592,teste,3000.0,2023-08-05,2023,AGO,-40,A VENCER,2023-08-05,2023,AGO,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
